In [1]:
import Tkinter as tk
import tkFileDialog as tkfd
import tkMessageBox as tkmb
import os
import sys
import json
import itertools
from DINGO.utils import read_config
from DINGO.base import DINGO
from pprint import pprint

loremipsum = ('Lorem ipsum dolor sit amet, consectetur adipiscing elit, '
              'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. '
              'Ut enim ad minim veniam, quis nostrud exercitation ullamco '
              'laboris nisi ut aliquip ex ea commodo consequat. '
              'Duis aute irure dolor in reprehenderit in voluptate velit '
              'esse cillum dolore eu fugiat nulla pariatur. '
              'Excepteur sint occaecat cupidatat non proident, '
              'sunt in culpa qui officia deserunt mollit anim id est laborum.')

shortlorem = 'Lorem ipsum dolor sit amet,'

class Extra_entry(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.scrollbar = tk.Scrollbar(self)
        self.scrollbar.grid(row=0, column=1)
        self.variables = []
        self.entries = []
        self.add_entry(0)
        
    def callback(self):
        for var in self.variables:
            if var.get() == '':
                idx = self.variables.index(var)
                self.variables.pop(idx)
                entry = self.entries.pop(idx)
                entry.destroy()
                
        
    def add_entry(self, newentry):
        self.variables.append(tk.StringVar(self))
        self.entries.append(tk.Entry(self, 
                                    textvariable=self.variables[newentry],
                                    validate='focusout',
                                    validatecommand=self.callback))
        self.entries[newentry].grid(column=0)

class DINGOFile(tk.Frame):
    def __init__(self, parent, default_filename=''):
        tk.Frame.__init__(self, parent)
        
        self.file_label = tk.Label(self, text=default_filename)
        self.file_label.grid(row=0, column=0)
        
        self.new = tk.Button(self, text='New', command=self.new_config(parent))
        
        self.open = tk.Button(self, text='Open', command=self.ask_open_json)
        self.open.grid(row=0, column=1)
        
        self.save = tk.Button(self, text='Save', command=self.save_json)
        self.save.grid(row=1, column=2)
        
    def new_config(self, parent):
        keys = []
        for tup in DINGO.default_expected_keys:
            if isinstance(tup[0],(unicode,str)):
                keys.append(tup[0])
            elif isinstance(tup[0],(tuple,list)):
                keys.append(tup[0][0])
        config = dict(zip(keys, itertools.repeat(None)))
        parent.update_options(config)
        
    def ask_open_json(self):
        self.filename = tkfd.askopenfilename(
            initialdir=os.getcwd(), 
            title='Select file', 
            filetypes=(('json files','*.json'),('all files','*.*')))
        try:
            config = read_config(self.filename)
        except IOError:
            config = None
        return config
        
    def save_json(self, adict):
        filename = self.filename.get()
        with open(filename, 'w') as f:
            json.dump(adict, f)
            
class DINGOOptions(tk.Frame):
    def __init__(self, parent, config=None):
        tk.Frame.__init__(self, parent)
        
        self.scrollbar = tk.Scrollbar(self)
        self.scrollbar.grid(row=0, column=1)
        
        self.listbox = tk.Listbox(self)
        self.listbox.grid(row=0, column=0)
        
    def make_options_widget(self, k, v):
        print('k: {}, v: {}'.format(k,v))
        if isinstance(v,(unicode,str)):
            pass
            
    def make_input_widget(self, aninput):
        pass

class HidePrint():
    def __enter__(self):
        self._orig_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._orig_stdout

class DINGOApp(tk.Tk):
    def __init__(self, filename=None, label='DINGO', master=None):
        if master is None:
            #tk.NoDefaultRoot()
            tk.Tk.__init__(self)
        else:
            tk.Tk.__init__(self, master=master)
        self.grid_columnconfigure(0,weight=1)
        self.grid_rowconfigure(0,weight=1)
        self.resizable(True, False)
        self.create_widgets(configpath=filename)
        self.grid_widgets()
        self.update_options(self.file_manip.config)
        self.mainloop()
#        DINGOApp.main()
    
    @classmethod
    def main(cls):
        tk.NoDefaultRoot()
        root = tk.Tk()
        app = cls(root)
        app.grid_columnconfigure(0,weight=1)
        app.grid_rowconfigure(0,weight=1)
        root.resizable(True, False)
        root.mainloop()
            
    def create_widgets(self, configpath=None):
        self.options = DINGOOptions(self)
        self.file_manip = DINGOFile(self)
        
        if configpath is not None:
            self.aDINGO = DINGO(configpath=configpath)
        if self.config is not None:
            try:
                self.aDINGO.check_input_fields('DINGOApp', self.config, DINGO.default_expected_keys)
            except Exception as err:
                DINGOApp.handle_create_error(err)
                
        self.run_plugin = tk.Entry(self, text='Linear')
        self.run_plugin_args = Extra_entry(self)
        self.runButton = tk.Button(self, text='Run', command=self.run_DINGO(configpath))
        self.quitButton = tk.Button(self, text='Quit', command=self.quit)
        
    def grid_widgets(self):
        options = dict(sticky='nsew', padx=4, pady=4)
        self.file_manip.grid(row=0, column=0, **options)
        self.options.grid(row=1, column=0, **options)
        self.run_plugin.grid(row=2, column=0, **options)
        self.run_plugin_args.grid(row=2, column=1, **options)
        self.runButton.grid(row=3, column=0, **options)
        self.quitButton.grid(row=3, column=1, **options)
        
    def update_options(self, config):
        for k,v in config.iteritems():
            pprint(self.__dict__, indent=2)
            self.options.make_options_widget(k,v)
            
    @staticmethod        
    def handle_create_error(err):
        tkmb.showerror('DINGO Creation Error', err)
    
    @staticmethod
    def handle_run_error(err):
        raise tkmb.showerror('DINGO Run Error', err)
        
    def run_DINGO_mock(self, configpath):
        print('I did not run DINGO')
        msg='I did not run DINGO'
        raise(Exception(msg))
            
    def run_DINGO(self, config):
        with HidePrint():
            self.aDINGO = DINGO(cfg=config)
        try:
            pass
            self.aDINGO.create_wf_from_config(cfg=config)
        except Exception as err:
            DINGOApp.handle_create_error(err)
            
        try:
            self.aDINGO.run(**runoptions)
        except Exception as err:
            DINGOApp.handle_run_error(err)
        

In [2]:
aconfigpath='/home/pirc/Desktop/five-phantom-trip/Boston/ProcessDTI.json'
DINGOApp()

{ '_tclCommands': ['tkerror', 'exit', '140145953030464destroy'],
  '_tkloaded': 1,
  'children': { '140147068795576': <__main__.DINGOFile instance at 0x7f7688407ab8>,
                '140147068795792': <__main__.DINGOOptions instance at 0x7f7688407b90>},
  'master': None,
  'options': <__main__.DINGOOptions instance at 0x7f7688407b90>,
  'tk': <tkapp object at 0x7f7645c27930>}
k: steps, v: None
{ '_tclCommands': ['tkerror', 'exit', '140145953030464destroy'],
  '_tkloaded': 1,
  'children': { '140147068795576': <__main__.DINGOFile instance at 0x7f7688407ab8>,
                '140147068795792': <__main__.DINGOOptions instance at 0x7f7688407b90>},
  'master': None,
  'options': <__main__.DINGOOptions instance at 0x7f7688407b90>,
  'tk': <tkapp object at 0x7f7645c27930>}
k: included_ids, v: None
{ '_tclCommands': ['tkerror', 'exit', '140145953030464destroy'],
  '_tkloaded': 1,
  'children': { '140147068795576': <__main__.DINGOFile instance at 0x7f7688407ab8>,
                '1401470687957

TypeError: __init__() got an unexpected keyword argument 'cfg'